In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
import malaya

model = malaya.translation.en_ms.transformer(model = 'base')

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [5]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
right = [unidecode(s) for s in ms]
left = [unidecode(s) for s in eng]

In [6]:
len(ms), len(eng)

(998, 998)

In [7]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 998/998 [03:13<00:00,  5.15it/s]


In [8]:
len(results)

998

In [9]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [10]:
refs = [filtered_right]
sys = filtered_left

In [11]:
r = bleu.corpus_score(sys, refs)

In [12]:
r.__dict__

{'name': 'BLEU',
 'score': 42.21071347388556,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '72.8/50.0/35.7/25.8 (BP = 0.986 ratio = 0.986 hyp_len = 21724 ref_len = 22027)',
 'bp': 0.986149111013918,
 'counts': [15812, 10365, 7044, 4839],
 'totals': [21724, 20727, 19730, 18733],
 'sys_len': 21724,
 'ref_len': 22027,
 'precisions': [72.78585895783465,
  50.00723693732812,
  35.701976685250884,
  25.83142048790904],
 'prec_str': '72.8/50.0/35.7/25.8',
 'ratio': 0.9862441549008035}

In [13]:
chrf.corpus_score(sys, refs)

chrF2++ = 66.28